# OilyGiant Model 

## Purpose

The purpose of this project is to find the best location for a our customer, OilyGiant, to place a new well for mining oil. We are given oil well parameters in three distinct regions, upon which we will use to create our linear regression model. The model will predict the volume of reserves in the new wells, and the region with the highest total profit will be chosen for the new well. 

## Read Data

In [ ]:
# Import Libraries
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error as mse, r2_score
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy import stats as st

In [ ]:
# Read datasets 
df0 = pd.read_csv('datasets/geo_data_0.csv')
df1 = pd.read_csv('datasets/geo_data_1.csv')
df2 = pd.read_csv('datasets/geo_data_2.csv')

In [ ]:
# Shape of datasets 
print(df0.shape)
print(df1.shape)
print(df2.shape)

(100000, 5)
(100000, 5)
(100000, 5)


In [ ]:
# Check for missing values
print(df0.isnull().sum())
print()
print(df1.isnull().sum())
print()
print(df2.isnull().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [ ]:
# Check for duplicates
print(df0.duplicated().sum())
print()
print(df1.duplicated().sum())
print()
print(df2.duplicated().sum())

0

0

0


In [ ]:
# correlation between features and target
print('First dataset:')
print(df0.corr()['product'])
print()

# correlation between features and target
print('Second dataset:')
print( df1.corr()['product'])
print()

# correlation between features and target
print('Third dataset:')
print(df2.corr()['product'])

First dataset:
f0         0.143536
f1        -0.192356
f2         0.483663
product    1.000000
Name: product, dtype: float64

Second dataset:
f0        -0.030491
f1        -0.010155
f2         0.999397
product    1.000000
Name: product, dtype: float64

Third dataset:
f0        -0.001987
f1        -0.001012
f2         0.445871
product    1.000000
Name: product, dtype: float64


Data did not need to be cleaned, as there were neither missing values nor duplicates found. This is crucial, as models cannot run with missing or duplicates. 

<div class="alert alert-success">
<b>Reviewer's comment</b>

Alright, the data was loaded and inspected!

</div>

## Prepare Data for Models

In [ ]:
print(df0.id.nunique())
print(df1.id.nunique())
print(df2.id.nunique())

99990
99996
99996


Most ID's are unique. 

In [ ]:
# First dataset
target_0 = df0['product']
features_0 = df0.drop(['product', 'id'], axis=1)

# Second dataset
target_1 = df1['product']
features_1 = df1.drop(['product', 'id'], axis=1)

# Third dataset
target_2 = df2['product']
features_2 = df2.drop(['product', 'id'], axis=1)


Dropped the categorical id column in order to run the models

<div class="alert alert-success">
<b>Reviewer's comment</b>

Good!

</div>

In [ ]:
# Random state variable
random_state = 19

In [ ]:
# Splitting data into train and valid sets, 75/25 

# First dataset
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=random_state)

# Second dataset
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=random_state)

# Third dataset
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=random_state)

<div class="alert alert-success">
<b>Reviewer's comment</b>

The data for each region was split into train and validation

</div>

In [ ]:
# Shape of results
print(features_train_0.shape)
print(target_train_0.shape)
print(features_valid_0.shape)
print(target_valid_0.shape)

(75000, 3)
(75000,)
(25000, 3)
(25000,)


## Model Training

In [ ]:
# Model Function
def linear_model(ft, tt, fv, tv, n): # features_train, target_train, features_valid, target_valid, dataset number
    
    model = LinearRegression()
    model.fit(ft, tt) # train model on training set
    predictions_valid = model.predict(fv) # get model predictions on validation set
    result = mse(tv, predictions_valid) ** 0.5 # calculate RMSE on validation set
           
    print('RMSE of the linear regression model on the validation set:', result)
    print('Dataset number :', (n))
    print('R2 score: ', r2_score(tv, predictions_valid))

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

It is not very useful to tune the `n_jobs` parameter of LinearRegression: it doesn't really affect the quality of the model, only the speed (and moreover, it only provides speedup on sufficiently large datasets, and our dataset is pretty small).
    
Other than that, your usage of the validation set and cross-validation is odd: first you tuned hyperparameters using the validation set (which is actually our test set in this case), and then used cross-validation to calculate 'average model evaluation score'. Check out [this guide](https://sebastianraschka.com/faq/docs/evaluate-a-model.html) on how to evaluate models.

</div>

<div class="alert alert-info">
  beleive i fixed this
</div>

<div class="alert alert-success">
<b>Reviewer's comment V2</b>

Alright!

</div>

In [ ]:
# Dataset 0
linear_model(features_train_0, target_train_0, features_valid_0, target_valid_0, 0)

RMSE of the linear regression model on the validation set: 37.489176404931385
Dataset number : 0
R2 score:  0.2800427586322962


In [ ]:
# Dataset 1
linear_model(features_train_1, target_train_1, features_valid_1, target_valid_1, 1)

RMSE of the linear regression model on the validation set: 0.8949747975866176
Dataset number : 1
R2 score:  0.9996221715647515


In [ ]:
# Dataset 2
linear_model(features_train_2, target_train_2, features_valid_2, target_valid_2, 2)

RMSE of the linear regression model on the validation set: 40.03911711877308
Dataset number : 2
R2 score:  0.19763128804301877


The linear model with the highest quality was with dataset 1. That model had a coefficient of determination, R^2, of .0999. This means the model almost perfectly predicts all the targets. This model also had the lowest RMSE. The models did not perform as well with datasets 0 and 2. 

<div class="alert alert-success">
<b>Reviewer's comment</b>

Great!

</div>

#### First Region

In [ ]:
# Model 0
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0) # train model on training set

# Predictions for model 0
predictions_0 = model_0.predict(features_valid_0)
predictions_0 = pd.Series(predictions_0)

# Merged dataframe of target and predictions 
merged_0 = pd.concat([target_valid_0.reset_index(drop=True), predictions_0], axis=1)
merged_0.columns = ['target', 'predictions']


## Second Region

In [ ]:
# Model 1
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1) # train model on training set

# Predictions for model 1
predictions_1 = model_1.predict(features_valid_1)
predictions_1 = pd.Series(predictions_1)

# Merged dataframe of target and predictions 
merged_1 = pd.concat([target_valid_1.reset_index(drop=True), predictions_1], axis=1)
merged_1.columns = ['target', 'predictions']

### Third Region

In [ ]:
# Model 2
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2) # train model on training set

predictions_2 = model_2.predict(features_valid_2)
predictions_2 = pd.Series(predictions_2)

merged_2 = pd.concat([target_valid_2.reset_index(drop=True), predictions_2], axis=1)
merged_2.columns = ['target', 'predictions']

Here, we trained our models with the various datasets to create the predictions on well volume. We then merged these predictions to their corresponding targets to create one dataframe. 

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

If we use the data the model was trained on in further steps, we're going to get overly optimistic results, because we're overestimating how good our models are (of course they perform better on the data they were trained on). So in the next steps we need to use predictions and targets for the validation set, not the whole dataset.

</div>

<div class="alert alert-info">
  fixed
</div>

<div class="alert alert-danger">
<s><b>Reviewer's comment V2</b>

Two problems:
    
1. For the third region predictions are still made for the whole dataset (`features_2`)
2. Predictions and targets are concatenated incorrectly (try looking at the result, for example, `merged_0` dataframe): the problem is that they have different indices. This causes problems down the line, as the predictions and targets don't match

</div>

<div class="alert alert-info">
  fixed
</div>

<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Great!

</div>

### Comparing Regions

In [ ]:
print('Mean Predictions 0:', predictions_0.mean(), '        ', 'Stdev Predictions 0:', predictions_0.std())
print('Mean Predictions 1:', predictions_1.mean(),'        ', 'Stdev Predictions 1:', predictions_1.std())
print('Mean Predictions 2:', predictions_2.mean(), '        ', 'Stdev Predictions 2:', predictions_2.std())

Mean Predictions 0: 92.4295169412194          Stdev Predictions 0: 23.254610378472815
Mean Predictions 1: 68.6810218520009          Stdev Predictions 1: 46.04297790672525
Mean Predictions 2: 94.91135196705525          Stdev Predictions 2: 19.797603324800953


The budget for development of 200 oil wells is 100 USD million.
One barrel of raw materials brings 4.5 USD of revenue The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels). It appears that Region 2 has the highest average predicted well volume reserve. 

#### Region 0

In [ ]:
# Comparing target and predictions of dataset 0 
merged_0.sum()

target         2.311242e+06
predictions    2.310738e+06
dtype: float64

In [ ]:
# Comparing target and predictions of dataset 0 
merged_0.describe().T

,count,mean,std,min,25%,50%,75%,max
target,25000.0,92.449669,44.183614,0.000000,56.610227,91.61242,128.522832,185.355615
predictions,25000.0,92.429517,23.254610,-9.764724,76.659319,92.45290,108.449070,180.227277


#### Region 1

In [ ]:
# Comparing target and predictions of dataset 1
merged_1.sum()

target         1.716792e+06
predictions    1.717026e+06
dtype: float64

In [ ]:
# Comparing target and predictions of dataset 1
merged_1.describe().T

,count,mean,std,min,25%,50%,75%,max
target,25000.0,68.671662,46.043907,0.000000,26.953261,57.085625,107.813044,137.945408
predictions,25000.0,68.681022,46.042978,-2.061922,28.391403,58.319042,109.280975,139.984148


#### Region 2

In [ ]:
# Comparing target and predictions of dataset 2
merged_2.sum()

target         2.376934e+06
predictions    2.372784e+06
dtype: float64

In [ ]:
# Comparing target and predictions of dataset 2
merged_2.describe().T

,count,mean,std,min,25%,50%,75%,max
target,25000.0,95.077342,44.699862,0.009204,59.634772,94.882039,130.685889,190.029838
predictions,25000.0,94.911352,19.797603,16.341091,81.549013,94.925208,108.410820,173.959528


Comparing targets and predictions across all datasets, we can see that our models were excellent. The mean an sum of our target and predictions columns are identical across all three datasets. The metrics that are different are the standard deviation, min, and max values. Again, Region 2 has the highest mean and total barrel reserve volume. 

### Break Even Point

In [ ]:
# Cost per barrel
cpb = 4.5 

# budget
budget = 100000000

# Number of wells
wells = 200

# Factor for unit of barrels
factor = 1000

unit_cost = 4500

# Budget per well
bpw = budget / wells

# Budget per barrel
bpb = bpw / unit_cost

# Break even reserve volume
break_even = budget / (cpb * factor)

# Minimum volume to break even
min_product = break_even / wells


In [ ]:
# Comparing mean volume with break even point
print('Volume in region 0 after break even: ', f'{merged_0.target.mean() - (min_product):,}', 'barrels')
print('Volume in region 1 after break even: ', f'{merged_1.target.mean() - (min_product):,}', 'barrels')
print('Volume in region 2 after break even: ', f'{merged_2.target.mean() - (min_product):,}', 'barrels')

Volume in region 0 after break even:  -18.661442080048303 barrels
Volume in region 1 after break even:  -42.4394489042288 barrels
Volume in region 2 after break even:  -16.033768694089943 barrels


Break even metric is compared with the mean of the complete regional datasets, comparing the target values. Here, we see that the mean targets are all lower than the minimum volume of oil needed to break even. This evaluation is misleading, as some wells will be far more profitable than other wells. Furthermore, it is in the interest of the company to exclusively pick wells with the highest profit, and avoid wells that do not break even. 

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

While break even point is calculated correctly, multiplying the mean by 1000 (you're basically calculating the amount of barrels in 1000 wells) and subtracting the break even point doesn't really make sense. Can you try comparing the mean predicted reserves to break even (in terms of the amount of product in just one well)?

</div>

<div class="alert alert-info">
  fixed
</div>

<div class="alert alert-success">
<b>Reviewer's comment V2</b>

Yep, the result is correct. I would not say that the evaluation is misleading: it just shows that if we selected the wells randomly, we would lose money. But you are correct that some wells are more profitable than others, so if we use our model to select the better wells, we can hope to make a profit :)

</div>

### Profit of Selected Region

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

I don't quite understand what you mean by 'general' profit, but this function is not correct. Revenue is correct, but for expenses we don't have any information about the cost of mining each barrel. We only have information about how much developing 200 wells costs.
    

</div>

<div class="alert alert-info">
  removed
</div>

<div class="alert alert-success">
<b>Reviewer's comment V2</b>

Ok!

</div>

In [ ]:
# Function to determine profit predictions to target
def profit(target, predictions, count):
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:count]
    return (unit_cost * selected.sum()) - budget


<div class="alert alert-success">
<b>Reviewer's comment</b>

This profit function is correct on the other hand!

</div>

Profit takes into consideration the budget of 200 wells at $100 million. 

### Profit per Top 200 Wells per Region

In [ ]:
print('Total profit from Region 0 :', '$', f'{profit(target_0, predictions_0, 200 ):,}')
print('Total profit from Region 1 :', '$', f'{profit(target_1, predictions_1, 200 ):,}')
print('Total profit from Region 2 :', '$', f'{profit(target_2, predictions_2, 200 ):,}')

Total profit from Region 0 : $ -16,617,291.92250064
Total profit from Region 1 : $ -37,881,648.63171914
Total profit from Region 2 : $ -16,543,855.132537901


<div class="alert alert-warning">
<b>Reviewer's comment V4</b>

For correct results here you'd need to use `merged_i` datasets as well, as `target_i` and `predictions_i` have different indices

</div>

Based on the the data of the top 200 wells of each region, Region 2 will bring the most profit. This data is based on target values of the largest 200 model predictions. 

<div class="alert alert-warning">
<b>Reviewer's comment</b>

As we're only allowed to take a random sample of 500 wells, and then select the best 200 out of those 500, it is highly unlikely that the overall top 200 wells in the region fall into this small sample, so I'm not sure how this is relevant.

</div>

<div class="alert alert-info">
  fixed
</div>

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

As noted above `profit_1` function is not correct

</div>

<div class="alert alert-info">
  removed
</div>

<div class="alert alert-success">
<b>Reviewer's comment V2</b>

Ok!

</div>

## Risk and Profit for each Region

### Confidence Intervals

In [ ]:
# Function for confidence interval
def confidence(sample): 
    print('Mean:', sample.mean())
    confidence_interval = st.t.interval(0.95, len(sample)-1, sample.mean(), sample.sem())
    print('95% confidence interval:', confidence_interval)

In [ ]:
confidence(predictions_0)

Mean: 92.4295169412194
95% confidence interval: (92.14124114410349, 92.71779273833532)


In [ ]:
confidence(predictions_1)

Mean: 68.6810218520009
95% confidence interval: (68.11025003765397, 69.25179366634782)


In [ ]:
confidence(predictions_2)

Mean: 94.91135196705525
95% confidence interval: (94.66593096078783, 95.15677297332267)


This confirms the earlier theory in the distribution of the well values among the regions. Region 2 values are slightly higher than Region 1. Region 0 is mostly distributed around smaller values of barrel reserve volume. 

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

I'm not sure I understand the point of the above two sections. Why would you need bootstrapping to calculate average reserve volume and its quantiles?

</div>

<div class="alert alert-info">
  removed
</div>

### Profit Risk

In [ ]:
# Function for determining profit, risk, and losses 
def profit_risk(target, predictions):
    state = np.random.RandomState(19)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        pred_subsample = predictions[target_subsample.index]
        values.append(profit(target_subsample, pred_subsample, 200))

    values = pd.Series(values)
    losses = (values < 0 ).mean() * 100
   

    print((f'{values.quantile(0.025):,}', f'{values.quantile(0.975):,}'))
    print('Mean Profit: ', f'{values.mean():,}')
    print('Losses : ', losses)   
    return
    

 

#### Region 0

In [ ]:
# Region 0
profit_risk(merged_0.target, merged_0.predictions)

('-682,314.259189359', '9,774,510.185401097')
Mean Profit:  4,533,870.619212871
Losses :  4.8


<div class="alert alert-info">
  Could use some help with this. looking through the documentation but unsure how to solve it or why it stopped working. 
</div>

<div class="alert alert-danger">
<s><b>Reviewer's comment V2</b>

Looks like this is due to the problem with concatenation, if you fix that, it should be fine

</div>

<div class="alert alert-info">
  fixed
</div>

<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Ok!

</div>

#### Region 1

In [ ]:
# Region 1
profit_risk(merged_1.target, merged_1.predictions)

('681,283.8236853313', '8,630,951.198799688')
Mean Profit:  4,945,452.475105005
Losses :  1.0999999999999999


#### Region 2

In [ ]:
# Region 2
profit_risk(merged_2.target, merged_2.predictions)

('-1,538,054.1441032141', '9,219,736.453033209')
Mean Profit:  3,756,892.022407412
Losses :  8.3


### Results

Based on the mean profit, Region 1 has the highest profit potential. Furthermore, Region 1 has the lowest chance of losses, at just over 1%. The 95% confidence zone of Region 1, has the highest range for profit. Regions 0 and  2 have 4.8 and 8.3 % chances of losses, respectively. This is supported by negative profit values seen in the 95% confidence interval. 

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

The function for finding profit distribution using bootstrapping is correct, but I have two objections about the calculations on this data:
    
1. A loss is defined as a negative profit, so why is it filtered as values lower than 22?
2. The task is to find 95% confidence interval for profit, not the 0.95 quantile :)
    
> these metrics may be misleading, as they are taking a sample of the 200 largest values of a sample of 500.
    
Why is that misleading? That's what our conditions are: we have the budget to make initial measurements at 500 locations, use our models to predict the amount of reserves contained in these 500 locations, and then develop the best 200.

</div>

<div class="alert alert-danger">
<b>Reviewer's comment V2</b>

The second problem is fixed, but for the first one it's still incorrect: the output from the boostrap is a list of profit values, not product values, so we literally just need to select the values lower than 0 :)

</div>

<div class="alert alert-info">
  fixed
</div>

<div class="alert alert-danger">
<s><b>Reviewer's comment V3</b>

Ok, one last thing: I'm not sure what changed, but now the profit values look incorrect, and it turns out that in the profit function there is an unnecessary multiplication by `factor`, to get the correct value the formula is `unit_cost * selected.sum() - budget`

</div>

<div class="alert alert-info">
  fixed
</div>

<div class="alert alert-success">
<b>Reviewer's comment V4</b>

Ok, great!

</div>

# Conclusion

Considering the three datasets, we will suggest OilGiant to start a new site in Region 1. Region 1 has the greatest profit margin, as the range of well reserve volume is better than those of the other regions, due to the higher lower bound. In addition, the chances of lossses in Region 1 is around 1%, which is extremely low. Therefore, the chances of randomly picking 200 sites that are extremely profitable will be more likely in Region 1. As we are focusing on as subsample of the 200 most profitable wells out of a 500 well sample, we limit our chance of loss, rather than just randomly picking wells. Overall, our model will perform well when predicting new sites to implement, and which wells are the most profitable, given the same features we have in our model. 

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

The choice of region 1 is correct and justified.
    
However, the following recommendation doesn't seem to make sense:
    
> However, if calculations were based on the the entire dataset, including all the wells of each region, Region 2 would be the recommendation.
    
I'm not sure what 'calculations based on the entire dataset' would mean here: suppose we could somehow make the initial measurements that would allow us to predict the amount of reserves in all wells in the region instead of just 500 locations, but that would increase the costs to astronomical amounts which we didn't account for :)

</div>

<div class="alert alert-info">
  addressed
</div>

<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Great!

</div>